# Run Struphy main file in notebook

In this tutorial we will learn about the Struphy main execution file `struphy/models/main/main.py`. This file is executed upon calling
```
    $ struphy run MODEL
```
from the console. We will

1. Import `struphy/models/main/main.py` and look at its functionality.
2. Import the parameters file `params_mhd_vlasov.yml` and change some parameters.
3. Understand the normalization of Struphy models (which units are used).
3. Run the model `LinearMHDVlasovCC` in the notebook (without invoking the console).

Let us start with step 1.

In [ ]:
from struphy.models.main import main

main?

The function `main.py` has the three mandatory arguments `model_name`, `parameters` and `path_out`. In this example, we shall simulate the current coupling hybrid model `LinearMHDVlasovCC`. The simulation results will be stored in the curnt working directory under the folder `struphy_run_main/`.

In [ ]:
import os

model_name = 'LinearMHDVlasovCC'

path_out = os.path.join(os.getcwd(), 'struphy_run_main/')

As a next step, we want to inspect available parameters files for our run. Template parameter files for each model are available in the struphy installation path (`struphy -p` in the console) un der the folder `io/inp/`. Let us check these out:

In [ ]:
import struphy

inp_path = os.path.join(struphy.__path__[0], 'io/inp')

os.listdir(inp_path)

The file `params_mhd_vlasov.yml` is the one for our run. Let us import it with the `yaml` package and print the obtained dictionary:

In [ ]:
params_path = os.path.join(inp_path, 'params_mhd_vlasov.yml')

import yaml

with open(params_path) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)
    
parameters

First, let us understand the units used in Struphy (model normalization). In the present example, the geometry is a `Cuboid` with specific left and right boundaries (and thus side length) in each of the three space directions:

In [ ]:
parameters['geometry']

The question arises in which units of length these numbers are expressed. From the console, the units could be checked by typing
```
    $ struphy units -i params_mhd_vlasov.yml LinearMHDVlasovCC
```
Here, you have to pass two informations, namely the parameter file and the model name. The latter is obvious because each Struphy model has its own specific normalization, stated in the model's documentation (and docstring). However, the units of each model can also be influenced by the user via the parameter file. Let us check the relevant section in the dictionary:

In [ ]:
parameters['units']

Indeed, the user can set

1. the unit of length $\hat x$ in meter
2. the unit of the magnetic field strength $\hat B$ in Tesla
3. the unit of the number density $\hat n$ in $10^{20}$ $m^{-3}$.

In the above example we have $\hat x \approx 0.023\,m$, $\hat B = 1\,T$ and $\hat n = 10^{20}$ $m^{-3}$. 

 All other units, such as for velocity $\hat v$ or time $\hat t$ etc., are derived from the specific normalization of the model. In particular, each model has built-in one of the following three choices for the velocity unit $\hat v$:

1. speed of light, $\hat v = c$
2. Alfvén speed of the bulk species, $\hat v = v_\textnormal{A, bulk} = \sqrt{\hat B^2 / (m_\textnormal{bulk} \hat n \mu_0)}$
3. Cyclotron speed of the bulk species, $\hat v = \hat x \Omega_\textnormal{c, bulk}/(2\pi) = \hat x\, q_\textnormal{bulk} \hat B /(m_\textnormal{bulk}2\pi)$

We can see that each of these velocity units can be defined in terms of the three units $\hat x$, $\hat B$ and $\hat n$ provided by the user. The associated time scale is then automatically given by
$$
 \hat t = \hat x / \hat v \,.
$$

 The velocity unit (in one-to-one correspondence with the `timescale`) and the `bulk_species` are fixed within each model, for instance:

In [ ]:
from struphy.models.hybrid import LinearMHDVlasovCC

print(LinearMHDVlasovCC.timescale())
print(LinearMHDVlasovCC.bulk_species())

The charge number `Z` and the mass number `A` of the bulk species can be set by the user via the parameter file:

In [ ]:
parameters['fluid'][LinearMHDVlasovCC.bulk_species()]['phys_params']

Please check out https://struphy.pages.mpcdf.de/struphy/sections/models.html#normalization for firther discussion on the units used in Struphy.In this tutorial, instead of the console, we can inspect the units of our run also directly in this notebook:

In [ ]:
units = LinearMHDVlasovCC.model_units(parameters, verbose=True)
units

The side lengths of the `Cuboid` in our example are:

In [ ]:
print('s1 = ', (parameters['geometry']['Cuboid']['r1'] - parameters['geometry']['Cuboid']['l1']) * units[0]['x'], 'm')
print('s2 = ', (parameters['geometry']['Cuboid']['r2'] - parameters['geometry']['Cuboid']['l2']) * units[0]['x'], 'm')
print('s3 = ', (parameters['geometry']['Cuboid']['r3'] - parameters['geometry']['Cuboid']['l3']) * units[0]['x'], 'm')

Let us get back to the parameter file and change some entries in the parameter dictionary before we run the model.

The end time of 50 is too long for our example and we wish to simulate more particles-per-cell than 200 to have a higher resolution. Let us change these two parameters and show the whole dictionary again.

In [ ]:
parameters['time']['Tend'] = 1.5

parameters['kinetic']['energetic_ions']['markers']['ppc'] = 400

parameters

We are now ready to call the Struphy main file. A tutorial of how to post process the generated simualtion data is available in the notebook `diagnostics_1_standard_plots`. 

In [ ]:
main(model_name, parameters, path_out)